# Lab7 - LiDAR data analysis and visualization with whitebox and leafmap**

LIDAR .LAS or .LAZ file can be downloaded from: 
https://apps.nationalmap.gov/lidar-explorer/#/



Create a new conda env to install required packages:

```bash
conda create -n geo python
conda activate geo
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install laspy[lazrs]
```

Uncomment the following line to install packages in Google Colab.

In [ ]:
# !pip install leafmap

In [ ]:
# !pip install laspy[lazrs]

## Import libraries

In [1]:
import os
import leafmap
import whitebox

c:\Users\C00553090\AppData\Local\miniconda3\envs\hypercoast\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


## Set up whitebox

In [2]:
wbt = whitebox.WhiteboxTools()
wbt.set_working_dir(os.getcwd())
wbt.set_verbose_mode(False)

0

## Download sample data

In [ ]:
filename = "USGS_LPC_Barataria_and_Jean_Lafitte_LiDAR_15RYN5357.laz"

## Read LAS/LAZ data

In [ ]:
# laz = leafmap.read_lidar(filename) is part of the leafmap library, 
# and it is used to read LiDAR data (LAS or LAZ format) into a Python object for further processing and visualization.
laz = leafmap.read_lidar(filename)

In [5]:
laz

<LasData(1.2, point fmt: <PointFormat(1, 0 bytes of extra dims)>, 37891 points, 3 vlrs)>

In [ ]:
# LAS 1.0 is basic and lacks many modern features.
# LAS 1.4 supports additional attributes like full waveform and higher precision.
# LAS 1.2 is a versatile format and widely supported by LiDAR processing tools.
str(laz.header.version)

'1.2'

## Upgrade file version

In [ ]:
#Some modern tools (like GIS or analysis software) may require LAS 1.4 to work with advanced datasets. 

las = leafmap.convert_lidar(laz, file_version="1.4")

In [21]:
str(las.header.version)

'1.4'

## Write LAS data

In [ ]:
# The command leafmap.write_lidar(las, "LA.las") is used to save LiDAR data to a file in the specified LAS or LAZ format. Here's a detailed breakdown:
leafmap.write_lidar(las, "LA.las")

## Histogram analysis

In [ ]:
# The command wbt.lidar_histogram("LA.las", "histogram.html") is part of the WhiteboxTools (WBT) library and is used to generate a histogram of the LiDAR data stored in the LAS file (LA.las). 
# it generates an interactive histogram of the attribute distributions in the LiDAR file LA.las and saves it as an HTML file (histogram.html). 
# This tool helps analyze and visualize the data's attributes, such as elevation or intensity, in a user-friendly format.

wbt.lidar_histogram("LA.las", "histogram.html")

0

## Visualize LiDAR data

In [11]:
leafmap.view_lidar("LA.las")

Widget(value='<iframe src="http://localhost:56176/index.html?ui=P_0x297f03af820_0&reconnect=auto" class="pyvis…

## Remove outliers

In [ ]:
# The command wbt.lidar_elevation_slice("LA.las", "LA_rm.las", minz=0, maxz=450) is used to filter LiDAR points based on their elevation (Z values). It keeps only the points within a specified elevation range and removes all others.
wbt.lidar_elevation_slice("LA.las", "LA_rm.las", minz=0, maxz=450)

0

## Visualize LiDAR data after removing outliers

In [13]:
leafmap.view_lidar("LA_rm.las", cmap="terrain")

Widget(value='<iframe src="http://localhost:56176/index.html?ui=P_0x297f03ae7a0_1&reconnect=auto" class="pyvis…

## Create DSM

In [ ]:
# The command is used to create a Digital Surface Model (DSM) from a LiDAR point cloud file.
wbt.lidar_digital_surface_model(
    "LA_rm.las", "dsm.tif", resolution=1.0, minz=0, maxz=450
)

0

In [ ]:
# The command is used to assign a coordinate system (CRS) to the file dsm.tif.
leafmap.add_crs("dsm.tif", epsg=26915)

## Visualize DSM

In [16]:
m = leafmap.Map()
m.add_raster("dsm.tif", palette="terrain", layer_name="DSM")
m

Map(center=[29.433113, -90.37282450000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

## Create DEM

In [ ]:
# Create a Digital Terrain Model (DTM) by removing objects like trees, buildings, or other above-ground features from a Digital Surface Model (DSM).
wbt.remove_off_terrain_objects("dsm.tif", "dem.tif", filter=25, slope=15.0)

0

## Visualize DEM

In [18]:
m.add_raster("dem.tif", palette="terrain", layer_name="DEM")
m

Map(bottom=13904831.0, center=[29.433113, -90.37282450000001], controls=(ZoomControl(options=['position', 'zoo…

## Create CHM

In [19]:
chm = wbt.subtract("dsm.tif", "dem.tif", "chm.tif")

In [20]:
m.add_raster("chm.tif", palette="gist_earth", layer_name="CHM")
m

Map(bottom=6952583.0, center=[29.433113, -90.37282450000001], controls=(ZoomControl(options=['position', 'zoom…